# Milestone 3 player Visualization

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import urllib.request
from PIL import Image
from plotly.subplots import make_subplots
import os

Frist we sort out the most improved player in the database and store it as the csv file.

In [ ]:
male_players_df = pd.read_csv('data/male_players.csv', usecols=['player_id', 'fifa_version', 'fifa_update','short_name', 'overall', 'age', 'club_name', 'player_face_url', 'value_eur'])

In [ ]:
male_players_df.sort_values(by=['fifa_version','fifa_update'], ascending=False, inplace=True)
male_players_df.drop_duplicates(subset=['player_id', 'fifa_version'], inplace=True,keep='first')
print(male_players_df.shape)

(196933, 9)


In [ ]:
import scipy.stats as stats
def player_classifier(overall_score,mean, std): # return player class based on overall score
    tvalue = stats.norm.cdf(overall_score, loc = mean, scale = std)
    threadshold_lt = {0.999:"world-top player", 0.99:"top player", 0.9:"great player", 
                      0.5:"above-average player", 0.1:"below-average player", 0:"bad player"}
    threadshold_key = sorted(threadshold_lt.keys())
    left = 0 
    right = len(threadshold_key) -1
    result = None
    # binary search
    while left <= right:
        mid = (left+right)//2
        if threadshold_key[mid] <= tvalue:
            result = threadshold_key[mid]
            left = mid + 1
        else:
            right = mid - 1
    
    return threadshold_lt[result]

stats_dict = male_players_df.groupby(['fifa_version'])['overall'].agg(['mean','std']).to_dict('index')

def map_player_classifier(row):
    mean = stats_dict[row['fifa_version']]['mean']
    std = stats_dict[row['fifa_version']]['std']
    return player_classifier(row['overall'], mean, std)

male_players_df['player_class'] = male_players_df.apply(map_player_classifier,axis=1)

In [ ]:
class_rank_df = male_players_df.groupby(['player_id'],as_index=False)['player_class'].agg(lambda x: np.unique(x).size).sort_values(by=['player_class','player_id'],ascending=False)
class_rank_df = class_rank_df[class_rank_df['player_class']>1]
print(f"The numbers of players who have class change in the 8 years are {class_rank_df.shape[0]}")

The numbers of players who have class change in the 8 years are 23143


We found that the maximum class change times among all players is 5.

In [ ]:
class_rank_df[class_rank_df['player_class']>4].to_csv('data/player_most_imporved.csv',index=False) #store the most improved players
tmp = class_rank_df[class_rank_df['player_class']>4]['player_id']
improved_players = male_players_df[male_players_df['player_id'].isin(tmp)]
improved_players.to_csv('data/improved_players_rank.csv',index=False) #store the rank of the most improved players

# Restore the full information of the most improved players

In [3]:
player_id = pd.read_csv('data/player_most_imporved.csv')
id_list = player_id['player_id'].tolist()

In [12]:
lines =[]
import csv
with open('data/male_players.csv', 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        #print(row)
        if int(row['player_id']) in id_list:
            #print(row)
            lines.append(row)

In [16]:
df = pd.DataFrame(lines)
df.sort_values(by=['fifa_version','fifa_update'], ascending=False, inplace=True)
df.drop_duplicates(subset=['player_id', 'fifa_version'], inplace=True,keep='first')


In [21]:
rank_df = pd.read_csv('data/improved_players_rank.csv')
rank_df.drop(columns=['short_name','overall','value_eur','age','club_name','player_face_url'],inplace=True)
df['player_id']=df['player_id'].astype(int)
df['fifa_version']=df['fifa_version'].astype(int)
df['fifa_update']=df['fifa_update'].astype(int)
out = pd.merge(df, rank_df, on=['player_id','fifa_version','fifa_update'], how='left')
out.to_csv('data/improved_players_full.csv',index=False) #store the full information of the most improved players

Then we store the full information of the most improved players into json file for better configuration of the website.

In [2]:
full_df = pd.read_csv('data/improved_players_full.csv') #read the full information of the most improved players

In [3]:
full_df

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,player_class
0,231747,/player/231747/kylian-mbappe/230009,23,9,2023-01-13,K. Mbappé,Kylian Mbappé Lottin,"ST, LW",91,95,...,63+3,67+3,63+3,54+3,54+3,54+3,63+3,18+3,https://cdn.sofifa.net/players/231/747/23_120.png,world-top player
1,212622,/player/212622/joshua-kimmich/230009,23,9,2023-01-13,J. Kimmich,Joshua Walter Kimmich,"CDM, RB, CM",89,90,...,86+3,84+3,83+3,81+3,81+3,81+3,83+3,21+3,https://cdn.sofifa.net/players/212/622/23_120.png,world-top player
2,239085,/player/239085/erling-haaland/230009,23,9,2023-01-13,E. Haaland,Erling Braut Haaland,ST,89,94,...,64+3,63+3,61+3,62+3,62+3,62+3,61+3,19+3,https://cdn.sofifa.net/players/239/085/23_120.png,world-top player
3,215698,/player/215698/mike-maignan/230009,23,9,2023-01-13,M. Maignan,Mike Maignan,GK,87,90,...,41+2,36+2,34+2,35+2,35+2,35+2,34+2,85+2,https://cdn.sofifa.net/players/215/698/23_120.png,world-top player
4,228702,/player/228702/frenkie-de-jong/230009,23,9,2023-01-13,F. de Jong,Frenkie de Jong,CM,87,92,...,84+3,83+3,82+3,78+3,78+3,78+3,82+3,18+3,https://cdn.sofifa.net/players/228/702/23_120.png,world-top player
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,228702,/player/228702/frenkie-de-jong/150059,15,59,2015-09-10,F. de Jong,Frenkie de Jong,CM,59,76,...,55,55,51,49,49,49,51,12,https://cdn.sofifa.net/players/228/702/15_120.png,below-average player
242,226754,/player/226754/ismael-bennacer/150059,15,59,2015-09-10,I. Bennacer,Ismaël Bennacer,CAM,54,74,...,44,45,41,38,38,38,41,14,https://cdn.sofifa.net/players/226/754/15_120.png,bad player
243,226226,/player/226226/giovani-lo-celso/150059,15,59,2015-09-10,G. Lo Celso,Giovani Lo Celso,LM,53,70,...,40,41,36,31,31,31,36,12,https://cdn.sofifa.net/players/226/226/15_120.png,bad player
244,228618,/player/228618/ferland-mendy/150059,15,59,2015-09-10,F. Mendy,Ferland Mendy,LB,51,62,...,46,50,51,54,54,54,51,12,https://cdn.sofifa.net/players/228/618/15_120.png,bad player


In [62]:
column_names = full_df.columns.tolist()

with open('data/column_names.txt', 'w') as file:
    for column_name in column_names:
        file.write(column_name + '\n')

In [63]:
value_columns = ['player_id','long_name', 'fifa_version', 'value_eur','player_positions',
                 'overall','potential','value_eur','wage_eur','age','height_cm','weight_kg',
                 'club_name','club_position','player_tags','player_traits',
                 'pace','shooting','passing','dribbling','defending','physic',
                 'player_face_url','player_class']

In [64]:
df_analysis = full_df[value_columns]
df_analysis['year'] = df_analysis['fifa_version'].apply(lambda x: int('20'+str(x)))

In [70]:
fill_value = {'pace':0,'shooting':0,'passing':0,'dribbling':0,'defending':0,'physic':0,
              'player_tags':'','player_traits':'','club_position':'','player_positions':''}
df_analysis.fillna(value=fill_value, inplace=True)
json_str = df_analysis.to_dict(orient='records')
with open('FootViz/public/improved_players.json', 'w') as file: #store the full information of the most improved players in json format
    json.dump(json_str, file)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_18752\3192247678.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analysis.fillna(value=fill_value, inplace=True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_18752\3192247678.py:3: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  json_str = df_analysis.to_dict(orient='records')


We reconstruct json file for the website usage.

## Data Visualization

In [ ]:
full_data = pd.read_csv('data/improved_players_full.csv')
fill_value = {'pace':0,'shooting':0,'passing':0,'dribbling':0,'defending':0,'physic':0,
              'player_tags':'','player_traits':'','club_position':'','player_positions':''}
full_data.fillna(value=fill_value, inplace=True)
id_list = full_data['player_id'].unique().tolist()

In [ ]:
# reconstruct the data for visualization
player_lt = []
def player_data(id):
    player = {}
    player['player_id'] = id_list[0]
    player['long_name'] = full_data[full_data['player_id'] == id]['long_name'].iloc[0]
    player['player_positions'] = full_data[full_data['player_id'] == id]['player_positions'].iloc[0]
    player['overall'] = full_data[full_data['player_id'] == id]['overall'].to_list()[::-1]
    player['potential'] = full_data[full_data['player_id'] == id]['potential'].to_list()[::-1]
    player['age'] = full_data[full_data['player_id'] == id]['age'].iloc[0]
    player['player_tags'] = full_data[full_data['player_id'] == id]['player_tags'].iloc[0]
    player['player_traits'] = full_data[full_data['player_id'] == id]['player_traits'].iloc[0]
    player['value_eur'] = full_data[full_data['player_id'] == id]['value_eur'].to_list()[::-1]
    player['wage_eur'] = full_data[full_data['player_id'] == id]['wage_eur'].to_list()[::-1]
    player['player_face_url'] = full_data[full_data['player_id'] == id]['player_face_url'].iloc[0]
    player['evolutionImageURL'] = 'evolution.png'
    for index,row in full_data[full_data['player_id'] == id].iterrows():
    #print(row)
        level = f'skill_20{row.fifa_version}'
        player[level] = [row.shooting, row.passing, row.dribbling, row.defending, row.physic, row.pace]
    timeline = {}
    for index,row in full_data[full_data['player_id'] == id].iterrows():
        if row.club_name not in timeline.keys():
            start_date = f'20{row.fifa_version}-01-01'
            end_date = f'20{row.fifa_version+1}-01-01'
            timeline[row.club_name] = [start_date, end_date]
        else:
            timeline[row.club_name][0] = f'20{row.fifa_version}-01-01'
    player['timeline'] = timeline
    return player
for id in id_list:
    player_lt.append(player_data(id))

In [ ]:
json_df = pd.DataFrame(player_lt)
json_df.fillna(value=0, inplace=True)
json_df = json_df.applymap(lambda x: [0,0,0,0,0] if x == 0 else x)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_22956\2091588952.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  json_df = json_df.applymap(lambda x: [0,0,0,0,0] if x == 0 else x)


Then we use plotly.js for visualization.

In [ ]:
os.makedirs('FootViz/public/playerface', exist_ok=True)

# Color mapping for player classes
axvspan_color = {"world-top player":'red', "top player":'orange', "great player":'coral', 
                      "above-average player":'lightsalmon', "below-average player":'peachpuff',"bad player":'linen'}
axvspan_color = {"world-top player":'orangered', "top player":'tomato', "great player":'coral', 
                      "above-average player":'lightsalmon', "below-average player":'peachpuff',"bad player":'linen'}

sub_df = full_data[full_data['player_id'] == id].iloc[::-1]

In [ ]:
def get_vis(id):
    sub_df = full_data[full_data['player_id'] == id].iloc[::-1]

    # Create a figure with Plotly
    fig = go.Figure()
    discount =3_500_000
    # Add line plot for overall score
    fig.add_trace(
        go.Scatter(
            x=[sub_df['fifa_version'].iloc[0]],
            y=[sub_df['overall'].iloc[0]],
            marker=dict(
            size=[sub_df['value_eur'].iloc[0] / discount],
            opacity=0.3  
        ),
            mode='lines+markers',
            name=f"Overall",
            line=dict(color='deepskyblue'),
            customdata=[sub_df['club_name'].iloc[0],sub_df['value_eur'].iloc[0], sub_df['league_name'].iloc[0]],
            hovertemplate=(
                "<b>Club name:</b> %{customdata[0]}<br>"
                "<b>League name:</b> %{customdata[2]}<br>"
                "<b>Overall:</b> %{y}<br>"
                "<b>Value in EUR</b>: %{customdata[1]:,}<br>"
                "<extra></extra>"  # This removes the trace name from the hover label
            )
        )
    )
    fig.add_trace(
        go.Scatter(
            x=[sub_df['fifa_version'].iloc[0]],
            y=[sub_df['potential'].iloc[0]],
            mode='lines+markers',
            name=f"Potential",
            line=dict(color='deepskyblue'),
            hovertemplate=(
                "<b>Potential:</b> %{y}<br>"
            )
        )
    )
    for player_class, color in axvspan_color.items():
        fig.add_trace(
            go.Scatter(
                x=[None], y=[None],
                mode='markers',
                marker=dict(size=10, color=color),
                legendgroup=player_class,
                showlegend=True,
                name=player_class,
                visible='legendonly'
            )
        )

    frames = []
    for i in range(1, len(sub_df) + 2):
        # Subset data for the current frame
        current_df = sub_df.iloc[:i]
        
        # Add the current frame data
        frame_data = [go.Scatter(
            x=current_df['fifa_version'],
            y=current_df['overall'],
            marker=dict(
                size=current_df['value_eur'] / discount,
                opacity=0.3 ,
            ),
            mode='lines+markers',
            line=dict(color='deepskyblue'),
            customdata=current_df[['club_name','value_eur','league_name']],
        )]
        frame_data.append(go.Scatter(
            x=current_df['fifa_version'],
            y=current_df['potential'],
            mode='lines+markers',
            line=dict(color='green')
        ))
        
        # Add colored spans for player classes up to the current frame
        annotations = []
        shapes = []
        #player_class = None
        #xleft = current_df['fifa_version'].min() - 0.5
        
        for j, row in current_df.iterrows():
            """
            next_player_class = row['player_class']
            #print(player_class)
            if (player_class is None):
                player_class = next_player_class
                continue
            elif (player_class != next_player_class):
                xright = row['fifa_version'] - 0.5
            elif (j == len(current_df) - 1) &(i == len(sub_df) + 1):
                xright = row['fifa_version'] + 5
            else:
                xright = row['fifa_version'] + 0.5
            """
            player_class = row['player_class']
            xleft = row['fifa_version'] - 0.5
            xright = row['fifa_version'] + 0.5
            shapes.append(dict(
                type="rect",
                xref="x",
                yref="paper",
                x0=xleft,
                x1=xright,
                y0=0,
                y1=1,
                fillcolor=axvspan_color[player_class],
                layer="below",
                line_width=0,
            ))
            frame_data.append(go.Scatter(
                x=[xleft, xright],
                y=[0,0],  # Dummy y-values, we won't use them
                #fillcolor=axvspan_color[player_class],
                fillcolor='orange',
            ))
            annotations.append(dict(
                x=(xleft + xright) / 2,
                y=max(sub_df['overall']),
                text=player_class,
                showarrow=False,
                yshift=10
            ))
            #xleft = xright
            #player_class = next_player_class
        
        frames.append(go.Frame(
            data=frame_data,
            layout=go.Layout( shapes=shapes)
        ))

    fig.frames = frames

    # Add player's face image as an annotation
    face_image = np.unique(sub_df['player_face_url'])[-1]
    req = urllib.request.Request(face_image, headers={'User-Agent': 'Mozilla/5.0'})
    image = Image.open(urllib.request.urlopen(req))
    path = f'FootViz/public/playerface/{np.unique(sub_df["long_name"])[0]}.png'
    image.save(path)
    image_url = face_image  # Using the URL directly for Plotly annotation
    #print(image_url)

    fig.add_layout_image(
        dict(
            source=path,
            xref="paper", yref="paper",
            x=0.9, y=0.1,
            sizex=0.15, sizey=0.15,
            xanchor="center", yanchor="middle"
        )
    )

    # Update layout for animation
    fig.update_layout(
        height=600,
        width=800,
        showlegend=True,
        title_text=f"{np.unique(sub_df['long_name'])[0]}",
        title_x = 0.5,
        title_y = 0.9,
        xaxis=dict(
            showgrid=False  
        ),
        yaxis=dict(
            showgrid=False  
        ),
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            buttons=[dict(label="Play",
                            method="animate",
                            args=[None, dict(frame=dict(duration=500, redraw=True), fromcurrent=True, mode='immediate')]),
                        ]
        )]
    )

    fig.update_xaxes(title_text="FIFA version")

    # Show plot
    #fig.show()

    fig.write_html(f"FootViz/public/plots/{np.unique(sub_df['long_name'])[0]}.html")

In [ ]:
from tqdm import tqdm
for id in tqdm(id_list):
    get_vis(id)

  0%|          | 0/31 [00:00<?, ?it/s]

100%|██████████| 31/31 [00:14<00:00,  2.16it/s]


Afer we finish every visualization of the player, we update the json file we needs for the website.

In [ ]:
json_df['evolutionImageURL'] = json_df['long_name'].apply(lambda x: f'plots/{x}.html')

In [ ]:
for index,row in json_df.iterrows():#reconstruct the timeline data for timeline and visualization
    new_timeline = []
    old_timeline = row['timeline']
    for club in row['timeline'].keys():
        one_team =  {}
        one_team['club_name'] = club
        one_team['start_date'] = old_timeline[club][0]
        one_team['end_date'] = old_timeline[club][1]
        new_timeline.append(one_team)
    json_df.at[index,'timeline'] = new_timeline

In [ ]:
json_str = json_df.to_dict(orient='records')
with open('FootViz/public/improved_players.json', 'w') as file:
    json.dump(json_str, file)